In [3]:
# import libraries

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Libraries imported!")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

# Review U.S. Census Data

In [19]:
df_mi = pd.read_csv("MichAgeSexPop.csv")
df_mi

,GEO_AREA_NAME,CITY_NAME,COUNTY_NAME,TOTAL_POP,TOTAL_POP_MALE,TOTAL_POP_FEMALE,TOTAL_POP_20_24,TOTAL_POP_25_29,TOTAL_POP_30_34,TOTAL_POP_35_39,TOTAL_POP_18_24,MEDIAN_AGE
0,"Dearborn city, Wayne County, Michigan",Dearborn,Wayne,94325,48569,45756,7222,7191,5646,5650,10541,33.0
1,"Southfield city, Oakland County, Michigan",Southfield,Oakland,73141,30925,42216,4649,4180,3314,4604,6111,44.7
2,"Lansing city, Ingham County, Michigan",Lansing,Ingham,113568,52686,60882,9274,11974,8115,7188,12205,33.1
3,"Sterling Heights city, Macomb County, Michigan",Sterling Heights,Macomb,132967,66262,66705,7873,10148,8732,9229,10028,39.4
4,"Livonia city, Wayne County, Michigan",Livonia,Wayne,93970,46458,47512,5356,7525,6581,6439,6845,40.6
5,"Rochester Hills city, Oakland County, Michigan",Rochester Hills,Oakland,74700,34961,39739,3904,3155,3620,6753,5372,41.1
6,"Canton charter township, Wayne County, Michigan",Canton,Wayne,93017,44665,48352,5107,6070,5023,5283,6805,40.5
7,"Flint city, Genesee County, Michigan",Flint,Genesee,95932,45062,50870,8298,8357,5949,4615,10990,34.7
8,"Troy city, Oakland County, Michigan",Troy,Oakland,84259,41232,43027,5344,5687,4802,4255,7281,41.8
9,"Grand Rapids city, Kent County, Michigan",Grand Rapids,Kent,200230,96945,103285,20774,22976,17251,13467,26960,30.8


In [21]:
# convert to integers
df_mi["TOTAL_POP_20_24"] = df_mi["TOTAL_POP_20_24"].astype(int)
df_mi["TOTAL_POP_25_29"] = df_mi["TOTAL_POP_25_29"].astype(int)
df_mi["TOTAL_POP_30_34"] = df_mi["TOTAL_POP_30_34"].astype(int)
df_mi["TOTAL_POP_35_39"] = df_mi["TOTAL_POP_35_39"].astype(int)
df_mi["TOTAL_POP_18_24"] = df_mi["TOTAL_POP_18_24"].astype(int)

# add column for total
df_mi["TOTAL_POP_20_34"] = df_mi["TOTAL_POP_20_24"] = df_mi["TOTAL_POP_25_29"] + df_mi["TOTAL_POP_30_34"]

# sort ages
df_mi.sort_values(by=["TOTAL_POP_20_34"], ascending=False, inplace=True)
df_mi.head(10)

,GEO_AREA_NAME,CITY_NAME,COUNTY_NAME,TOTAL_POP,TOTAL_POP_MALE,TOTAL_POP_FEMALE,TOTAL_POP_20_24,TOTAL_POP_25_29,TOTAL_POP_30_34,TOTAL_POP_35_39,TOTAL_POP_18_24,MEDIAN_AGE,TOTAL_POP_20_34
19,"Detroit city, Wayne County, Michigan",Detroit,Wayne,672681,316529,356152,102047,60590,41457,38441,69698,35.0,102047
9,"Grand Rapids city, Kent County, Michigan",Grand Rapids,Kent,200230,96945,103285,40227,22976,17251,13467,26960,30.8,40227
14,"Ann Arbor city, Washtenaw County, Michigan",Ann Arbor,Washtenaw,121885,59821,62064,21316,13090,8226,7791,39166,26.9,21316
18,"Warren city, Macomb County, Michigan",Warren,Macomb,134583,64622,69961,20671,11638,9033,8898,12528,40.4,20671
2,"Lansing city, Ingham County, Michigan",Lansing,Ingham,113568,52686,60882,20089,11974,8115,7188,12205,33.1,20089
3,"Sterling Heights city, Macomb County, Michigan",Sterling Heights,Macomb,132967,66262,66705,18880,10148,8732,9229,10028,39.4,18880
16,"Clinton charter township, Macomb County, Michigan",Clinton Township,Macomb,100798,47800,52998,15610,9475,6135,6744,8130,39.3,15610
7,"Flint city, Genesee County, Michigan",Flint,Genesee,95932,45062,50870,14306,8357,5949,4615,10990,34.7,14306
4,"Livonia city, Wayne County, Michigan",Livonia,Wayne,93970,46458,47512,14106,7525,6581,6439,6845,40.6,14106
0,"Dearborn city, Wayne County, Michigan",Dearborn,Wayne,94325,48569,45756,12837,7191,5646,5650,10541,33.0,12837


# Examine Cities on Map

In [23]:
# import packages

import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

# All requested packages already installed.



In [24]:
# The Rocket Ferndale

address = "Ferndale, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Ferndale are {}, {}.".format(latitude, longitude))

# The Rocket Ypsilanti

address = "Ypsilanti, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Ypsilanti are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ferndale are 42.4605917, -83.1346478.
The geographical coordinates of Ypsilanti are 42.24253, -83.6171426940706.


In [25]:
# look at top five cities

# Detroit

address = "Detroit, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Detroit are {}, {}.".format(latitude, longitude))

# Grand Rapids

address = "Grand Rapids, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Grand Rapids are {}, {}.".format(latitude, longitude))

# Ann Arbor

address = "Ann Arbor, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Ann Arbor are {}, {}.".format(latitude, longitude))

# Warren

address = "Warren, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Warren are {}, {}.".format(latitude, longitude))

# Lansing

address = "Lansing, MI"

geolocator = Nominatim(user_agent="mi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Lansing are {}, {}.".format(latitude, longitude))

The geographical coordinates of Detroit are 42.3315509, -83.0466403.
The geographical coordinates of Grand Rapids are 42.9632405, -85.6678639.
The geographical coordinates of Ann Arbor are 42.2681569, -83.7312291.
The geographical coordinates of Warren are 42.4932575, -83.0062746.
The geographical coordinates of Lansing are 42.7337712, -84.5553805.


In [26]:
latlon = [(42.3315509, -83.0466403),\
          (42.9632405, -85.6678639),\
          (42.2681569, -83.7312291),\
          (42.7337712, -84.5553805),\
          (42.4932575, -83.0062746),\
          (42.4605917, -83.1346478),\
          (42.24253, -83.6171426940706)]

mapit = folium.Map(location=[42.460591, -83.1346478], zoom_start=10)

for coord in latlon:
    folium.CircleMarker(location=[coord[0], coord[1]], color="red", fill_color="red", radius=8, fill=True, fill_opacity=0.5).add_to(mapit)

mapit